In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# imports to visualize tree
from sklearn import tree as treeViz
import graphviz
# import pydotplus
from IPython.display import display

In [17]:
# load data
%run preprocess.ipynb
X_train, y_train, X_val, y_val, X_test, y_test = get_processed_data()

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(19821, 58)
(11894, 58)
(7929, 58)
targets: [[-0.14581653]
 [-0.15441734]
 [ 0.11220795]
 ...
 [ 0.09500632]
 [-0.25023043]
 [-0.1888206 ]]
data: [[-1.60772590e+00 -4.74451259e-01  1.78983723e-03 ...  5.47157435e-01
  -1.19274310e+00  2.67809642e-01]
 [ 1.23048186e+00  9.19437671e+00 -7.39034343e-02 ...  2.44637285e-01
  -1.56994907e+00 -8.70560322e-02]
 [ 7.57447233e-01 -3.51335557e-01  3.25676620e-03 ...  1.23781665e+00
  -2.21638357e-01  1.07797467e+00]
 ...
 [ 1.23048186e+00 -7.27050716e-01  2.79063632e-02 ...  1.23781665e+00
  -2.21638357e-01  1.07797467e+00]
 [ 2.84412606e-01  3.28703405e-02 -9.31336282e-03 ... -3.05226384e-01
  -1.13121305e+00 -6.47252789e-01]
 [-1.88622020e-01 -6.59124812e-01  1.32395104e-02 ... -2.69076177e-01
   8.37748634e-01 -6.89658121e-01]]
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFr

In [18]:
# Creating a DecisionTreeRegressor model
from sklearn import tree
from sklearn.metrics import mean_squared_error
regression_tree = tree.DecisionTreeRegressor(criterion="friedman_mse", splitter="best", max_depth=19, min_samples_split=2, min_samples_leaf=1)
regression_tree.fit(X_train, y_train)
# Make predictions on the test set
y_pred = regression_tree.predict(X_val)
# Evaluate the model
mse = mean_squared_error(y_val, y_pred)

# Print the training and validation scores (accuracy)
print("Training Accuracy:", regression_tree.score(X_train, y_train))
print("Validation Accuracy:", regression_tree.score(X_val, y_val))
print("Validation Mean Squared Error:", mse)



Training Accuracy: 0.9276312287879077
Validation Accuracy: -0.15109389063595713
Validation Mean Squared Error: 3.0213350227920515


In [19]:
print("Mean Squared Error:", mse)
print("R2 Score:", regression_tree.score(X_val, y_val))
print("Feature Importances:", regression_tree.feature_importances_)
print("Number of Outputs:", regression_tree.n_outputs_)
print("Number of Nodes:", regression_tree.tree_.node_count)
print("Max Depth:", regression_tree.tree_.max_depth)
print("Min Samples Split:", regression_tree.min_samples_split)
print("Min Samples Leaf:", regression_tree.min_samples_leaf)

Mean Squared Error: 3.0213350227920515
R2 Score: -0.15109389063595713
Feature Importances: [1.50137280e-02 4.67857997e-02 1.29760540e-01 0.00000000e+00
 1.42208512e-02 8.35919640e-03 2.15104433e-03 8.70444509e-03
 5.13051697e-03 1.75312721e-01 2.96461761e-03 2.12229698e-03
 2.82961401e-04 6.11890619e-03 2.35195509e-03 2.58728395e-04
 6.78870737e-06 9.30570308e-04 1.04266634e-02 7.12170482e-03
 3.24082941e-03 6.20742376e-03 2.73772474e-02 5.86416584e-03
 3.24732641e-02 3.06430913e-02 2.19653553e-02 7.06829368e-03
 5.39853007e-02 2.83243881e-04 7.01674030e-04 2.48195227e-03
 8.22801577e-04 6.94336716e-05 1.59206566e-04 1.36927791e-04
 1.15405689e-03 4.77571731e-02 1.72398584e-02 1.28248160e-02
 2.46322458e-02 3.62575041e-03 1.24266961e-01 6.27589384e-03
 8.98295604e-03 8.96763666e-03 1.90100167e-03 7.86606832e-03
 8.25037001e-03 6.70213253e-03 1.36005292e-03 2.17122414e-02
 3.93447910e-03 9.13040888e-03 2.24406593e-02 1.63537536e-02
 2.72390286e-03 1.03933345e-02]
Number of Outputs: 1
Nu

In [20]:
def hyperparameter_tune_grid_search(criterion, min_samples_split, splitter, max_depth, X_train, y_train, X_val, y_val):
    """Tune hyperparameters using grid search. Return the best score and the best parameters.
    param criterion: list of strings, the function to measure the quality of a split
    param min_samples_split: list of integers, the minimum number of samples required to split an internal node
    param splitter: list of strings, the strategy used to choose the split at each node
    param max_depth: list of integers, the maximum depth of the tree
    param X_train: array-like, the training input samples
    param y_train: array-like, the target values"""
    best_score = 0
    best_params = {}
    for c in criterion:
        for mss in min_samples_split:
            for s in splitter:
                for md in max_depth:
                    regression_tree = tree.DecisionTreeRegressor(criterion=c, splitter=s, max_depth=md, min_samples_split=mss, min_samples_leaf=1)
                    regression_tree.fit(X_train, y_train)
                    score = regression_tree.score(X_val, y_val)
                    mse = mean_squared_error(y_val, y_pred)
                    if score > best_score:
                        best_score = score
                        best_params = {"criterion": c, "min_samples_split": mss, "splitter": s, "max_depth": md}
    return best_score, best_params

In [21]:
hyperparameter_tune_grid_search(['friedman_mse', 'squared_error'], [3, 5, 8], ["best", "random"], [5, 10, 15, 20, 50], X_train, y_train, X_val, y_val)

(0.008252970958679273,
 {'criterion': 'squared_error',
  'min_samples_split': 5,
  'splitter': 'random',
  'max_depth': 5})

In [ ]:
scores = []
mse_list = []
max_depths = []
min_samples_splits = []
for i in range(1, 100):
    for j in range(2, 100,5):
        regression_tree = tree.DecisionTreeRegressor(criterion="friedman_mse", splitter="best", max_depth=i, min_samples_split=j, min_samples_leaf=1)
        regression_tree.fit(X_train, y_train)
        y_pred = regression_tree.predict(X_val)
        score = regression_tree.score(X_val, y_val)
        mse = mean_squared_error(y_val, y_pred)
        scores.append(score)
        mse_list.append(mse)
        max_depths.append(i)
        min_samples_splits.append(j)


# plot a 3d scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(max_depths, min_samples_splits, scores)
ax.set_xlabel('Max Depth')
ax.set_ylabel('Min Samples Split')
ax.set_zlabel('Score')
plt.show()

# plot a 3d scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(max_depths, min_samples_splits, mse_list)
ax.set_xlabel('Max Depth')
ax.set_ylabel('Min Samples Split')
ax.set_zlabel('Mean Squared Error')
plt.show()


